# 🌟 완벽한 세상: 정규분포와 Z-검정

## 📖 이야기의 시작

1900년대 초, 통계학자들은 **완벽한 세상**을 꿈꿨습니다. 🌍✨

그들이 꿈꾸던 세상은 이런 곳이었습니다:
- 모든 데이터는 아름다운 **정규분포**를 따른다
- **모집단의 표준편차 σ**는 이미 알려져 있다
- 표본 크기는 충분히 크다 (n ≥ 30)

이런 완벽한 조건에서는 **Z-분포**를 사용하여 모든 통계적 검정을 완벽하게 수행할 수 있었습니다. 하지만... 과연 현실은 어떨까요? 🤔

---

## 🎯 학습 목표

이 노트북을 통해 다음을 학습합니다:
1. 정규분포의 특성과 중심극한정리
2. Z-분포와 표준화의 개념
3. Z-검정의 원리와 수행 방법
4. 신뢰구간의 의미와 해석
5. 완벽한 세상의 한계점

In [1]:
# 필수 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from scipy import stats
from ipywidgets import interact, widgets, IntSlider, FloatSlider
import warnings

warnings.filterwarnings('ignore')

# 시각화 스타일 설정
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# 한글 폰트 설정 (필요시)
plt.rcParams['font.family'] = 'DejaVu Sans'

print("📚 모든 라이브러리가 성공적으로 로드되었습니다!")
print("🎉 완벽한 세상으로의 여행을 시작합니다!")

📚 모든 라이브러리가 성공적으로 로드되었습니다!
🎉 완벽한 세상으로의 여행을 시작합니다!


## 🌈 1. 정규분포: 자연의 가장 아름다운 곡선

정규분포는 **가우스 분포**라고도 불리며, 자연 현상에서 가장 자주 관찰되는 분포입니다.

### 📊 정규분포의 특성
- **대칭성**: 평균을 중심으로 완벽하게 대칭
- **종 모양**: 아름다운 bell curve
- **68-95-99.7 규칙**: 
  - 68%의 데이터가 μ ± σ 범위에
  - 95%의 데이터가 μ ± 2σ 범위에
  - 99.7%의 데이터가 μ ± 3σ 범위에

### 🔢 수학적 표현

정규분포 N(μ, σ²)의 확률밀도함수:

$$f(x) = \frac{1}{\sigma\sqrt{2\pi}} e^{-\frac{(x-\mu)^2}{2\sigma^2}}$$

여기서:
- μ: 평균 (mean)
- σ: 표준편차 (standard deviation)
- σ²: 분산 (variance)

In [2]:
# 🎨 인터랙티브 정규분포 시각화
def plot_normal_distribution(mu=0, sigma=1, show_areas=True):
    """정규분포를 시각화하는 함수"""
    
    # 고정된 범위 설정 (더 넓은 범위로 설정)
    x_min, x_max = -15, 15
    y_min, y_max = 0, 0.5
    
    x = np.linspace(x_min, x_max, 1000)
    y = stats.norm.pdf(x, mu, sigma)
    
    # 플롯 생성
    fig = go.Figure()
    
    # 정규분포 곡선
    fig.add_trace(go.Scatter(
        x=x, y=y,
        mode='lines',
        name=f'N({mu}, {sigma}²)',
        line=dict(width=3, color='blue')
    ))
    
    if show_areas:
        # 68% 영역 (μ ± σ)
        x_68 = x[(x >= mu - sigma) & (x <= mu + sigma)]
        y_68 = stats.norm.pdf(x_68, mu, sigma)
        if len(x_68) > 0:
            fig.add_trace(go.Scatter(
                x=np.concatenate([x_68, x_68[::-1]]),
                y=np.concatenate([y_68, np.zeros(len(y_68))]),
                fill='toself',
                fillcolor='rgba(255, 0, 0, 0.3)',
                line=dict(width=0),
                name='68% (μ ± σ)',
                showlegend=True
            ))
        
        # 95% 영역 (μ ± 2σ)
        x_95_left = x[(x >= mu - 2*sigma) & (x < mu - sigma)]
        y_95_left = stats.norm.pdf(x_95_left, mu, sigma)
        x_95_right = x[(x > mu + sigma) & (x <= mu + 2*sigma)]
        y_95_right = stats.norm.pdf(x_95_right, mu, sigma)
        
        for x_95, y_95, name in [(x_95_left, y_95_left, '95% 영역 (좌)'), 
                                 (x_95_right, y_95_right, '95% 영역 (우)')]:
            if len(x_95) > 0:
                fig.add_trace(go.Scatter(
                    x=np.concatenate([x_95, x_95[::-1]]),
                    y=np.concatenate([y_95, np.zeros(len(y_95))]),
                    fill='toself',
                    fillcolor='rgba(0, 255, 0, 0.2)',
                    line=dict(width=0),
                    name='95% (μ ± 2σ)' if 'left' in name else '',
                    showlegend='left' in name
                ))
    
    # 평균선 추가
    fig.add_vline(x=mu, line_dash="dash", line_color="red", 
                  annotation_text=f"μ = {mu}")
    
    # 레이아웃 설정 - 축 범위 고정
    fig.update_layout(
        title=f'정규분포 N({mu}, {sigma}²) - 완벽한 세상의 기초',
        xaxis_title='값 (X)',
        yaxis_title='확률밀도',
        template='plotly_white',
        height=500,
        showlegend=True,
        xaxis=dict(range=[x_min, x_max]),  # X축 고정
        yaxis=dict(range=[y_min, y_max])   # Y축 고정
    )
    
    return fig

# 인터랙티브 위젯으로 정규분포 탐색
@interact(mu=FloatSlider(min=-5, max=5, step=0.5, value=0, description='평균 (μ)'),
          sigma=FloatSlider(min=0.5, max=3, step=0.1, value=1, description='표준편차 (σ)'),
          show_areas=widgets.Checkbox(value=True, description='영역 표시'))
def interactive_normal(mu, sigma, show_areas):
    # 기존 출력 클리어
    from IPython.display import clear_output
    clear_output(wait=True)
    
    fig = plot_normal_distribution(mu, sigma, show_areas)
    
    # 주요 통계량 출력
    print(f"""

💡 핵심 통찰:

📊 평균 (μ): {mu}
📏 표준편차 (σ): {sigma}
📐 분산 (σ²): {sigma**2:.2f}
🎯 68% 범위: [{mu-sigma:.2f}, {mu+sigma:.2f}]
🎯 95% 범위: [{mu-2*sigma:.2f}, {mu+2*sigma:.2f}]
🎯 99.7% 범위: [{mu-3*sigma:.2f}, {mu+3*sigma:.2f}]

""")
    
    fig.show()
    
    # 명시적으로 None 반환하여 추가 출력 방지
    return None

interactive(children=(FloatSlider(value=0.0, description='평균 (μ)', max=5.0, min=-5.0, step=0.5), FloatSlider(v…

## 🔄 2. 중심극한정리: 마법 같은 정리

중심극한정리(Central Limit Theorem)는 통계학의 가장 아름다운 정리 중 하나입니다.

### 📜 중심극한정리의 내용

> **어떤 분포**를 따르는 모집단이라도, 표본 크기가 충분히 클 때 (일반적으로 n ≥ 30), **표본평균들의 분포**는 정규분포에 가까워진다.

수학적으로 표현하면:

$$\bar{X} \sim N\left(\mu, \frac{\sigma^2}{n}\right)$$

또는 표준화하면:

$$Z = \frac{\bar{X} - \mu}{\frac{\sigma}{\sqrt{n}}} \sim N(0, 1)$$

### 💫 중심극한정리의 마법
- 원래 분포가 **어떤 모양**이든 상관없다!
- 균등분포, 지수분포, 베르누이분포... 모두 OK!
- 표본 크기가 클수록 더 정확해진다

### 📈 Q-Q Plot이란?

**Q-Q Plot**(Quantile-Quantile Plot)은 두 분포의 분위수(quantile)를 비교하는 시각화 도구입니다.

#### 🔍 Q-Q Plot 읽는 법
- **X축**: 이론적 정규분포의 분위수
- **Y축**: 실제 데이터의 분위수 (표준화됨)
- **빨간 대각선**: 완벽한 정규분포일 때의 이상적 선 (y = x)

#### 📊 해석 방법
✅ **점들이 대각선에 가까움** → 정규분포에 가까움  
❌ **점들이 대각선에서 벗어남** → 정규분포에서 벗어남

- **점들이 S자 곡선**: 치우침(skewness) 존재
- **점들이 위로 볼록**: 두꺼운 꼬리(heavy tail)
- **점들이 아래로 볼록**: 얇은 꼬리(light tail)

중심극한정리가 잘 작동한다면, 표본평균들의 Q-Q plot에서 점들이 빨간 대각선에 가깝게 분포해야 합니다!

In [3]:
# 🎭 중심극한정리 시뮬레이션
def demonstrate_central_limit_theorem(distribution='uniform', n_samples=30, n_simulations=1000):
    """중심극한정리를 시연하는 함수"""
    
    np.random.seed(42)  # 재현가능한 결과를 위한 시드 설정
    
    # 다양한 원분포에서 데이터 생성
    if distribution == 'uniform':
        population_data = np.random.uniform(0, 10, 10000)
        dist_name = '균등분포 U(0,10)'
    elif distribution == 'exponential':
        population_data = np.random.exponential(2, 10000)
        dist_name = '지수분포 Exp(λ=0.5)'
    elif distribution == 'skewed':
        population_data = np.random.gamma(2, 2, 10000)
        dist_name = '감마분포 Γ(2,2)'
    elif distribution == 'normal':
        population_data = np.random.normal(5, 2, 10000)
        dist_name = '정규분포 N(5,2²)'
    else:  # bimodal
        data1 = np.random.normal(3, 1, 5000)
        data2 = np.random.normal(7, 1, 5000)
        population_data = np.concatenate([data1, data2])
        dist_name = '이봉분포 (Bimodal)'
    
    # 표본평균들 수집
    sample_means = []
    for _ in range(n_simulations):
        sample = np.random.choice(population_data, n_samples, replace=True)
        sample_means.append(np.mean(sample))
    
    sample_means = np.array(sample_means)
    
    # 이론적 값들 계산
    pop_mean = np.mean(population_data)
    pop_std = np.std(population_data)
    theoretical_mean = pop_mean
    theoretical_std = pop_std / np.sqrt(n_samples)
    
    # 시각화
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=['1. 원래 모집단 분포', '2. 표본평균들의 분포', 
                       '3. 정규분포와 비교', '4. Q-Q 플롯'],
        specs=[[{'type': 'histogram'}, {'type': 'histogram'}],
               [{'type': 'scatter'}, {'type': 'scatter'}]]
    )
    
    # 1. 원래 분포 - 축 범위 고정
    fig.add_trace(
        go.Histogram(x=population_data[:1000], nbinsx=50, name='원분포', 
                    marker_color='lightblue', opacity=0.7),
        row=1, col=1
    )
    
    # 2. 표본평균들의 분포 - 축 범위 고정
    fig.add_trace(
        go.Histogram(x=sample_means, nbinsx=50, name='표본평균들', 
                    marker_color='lightgreen', opacity=0.7),
        row=1, col=2
    )
    
    # 3. 이론적 정규분포와 비교
    x_theory = np.linspace(0, 12, 100)  # 고정된 범위
    y_theory = stats.norm.pdf(x_theory, theoretical_mean, theoretical_std)
    
    # 히스토그램을 확률밀도로 변환
    counts, bins = np.histogram(sample_means, bins=50, density=True, range=(0, 12))
    bin_centers = (bins[:-1] + bins[1:]) / 2
    
    fig.add_trace(
        go.Bar(x=bin_centers, y=counts, name='실제 분포', 
              marker_color='lightcoral', opacity=0.7),
        row=2, col=1
    )
    
    fig.add_trace(
        go.Scatter(x=x_theory, y=y_theory, mode='lines', name='이론적 정규분포',
                  line=dict(color='red', width=3)),
        row=2, col=1
    )
    
    # 4. Q-Q 플롯 - 축 범위 고정
    theoretical_quantiles = stats.norm.ppf(np.linspace(0.01, 0.99, len(sample_means)))
    sample_quantiles = np.sort(sample_means)
    
    # 표준화
    sample_quantiles_std = (sample_quantiles - np.mean(sample_quantiles)) / np.std(sample_quantiles)
    
    fig.add_trace(
        go.Scatter(x=theoretical_quantiles, y=sample_quantiles_std, mode='markers',
                  name='Q-Q 점들', marker=dict(color='blue', size=4)),
        row=2, col=2
    )
    
    # 이상적인 선 (y=x)
    line_range = [-3, 3]  # 고정된 범위
    fig.add_trace(
        go.Scatter(x=line_range, y=line_range, mode='lines',
                  name='이상적 선', line=dict(color='red', dash='dash')),
        row=2, col=2
    )
    
    # 레이아웃 업데이트 - 축 범위 고정
    fig.update_layout(
        title=f'중심극한정리 시연: {dist_name} → 정규분포 (n={n_samples})',
        height=800,
        showlegend=True
    )
    
    # 각 서브플롯의 축 범위 고정
    fig.update_xaxes(range=[0, 12], row=1, col=1)  # 원분포 X축
    fig.update_yaxes(range=[0, 600], row=1, col=1)  # 원분포 Y축
    
    fig.update_xaxes(range=[0, 12], row=1, col=2)  # 표본평균 X축
    fig.update_yaxes(range=[0, 600], row=1, col=2)  # 표본평균 Y축
    
    fig.update_xaxes(range=[0, 12], row=2, col=1)  # 비교 X축
    fig.update_yaxes(range=[0, 2], row=2, col=1)   # 비교 Y축
    
    fig.update_xaxes(range=[-3, 3], row=2, col=2)  # Q-Q X축
    fig.update_yaxes(range=[-3, 3], row=2, col=2)  # Q-Q Y축
    
    # Q-Q 플롯 분석을 위한 데이터 반환
    qq_data = {
        'theoretical_quantiles': theoretical_quantiles,
        'sample_quantiles_std': sample_quantiles_std
    }
    
    return fig, sample_means, theoretical_mean, theoretical_std, qq_data

def analyze_qq_plot(theoretical_quantiles, sample_quantiles_std):
    """Q-Q 플롯을 분석하여 정규성과 패턴을 평가"""
    
    # 상관계수 계산 (선형성 측정)
    correlation = np.corrcoef(theoretical_quantiles, sample_quantiles_std)[0, 1]
    
    # 선형회귀로 기울기와 절편 계산
    slope, intercept = np.polyfit(theoretical_quantiles, sample_quantiles_std, 1)
    
    # 잔차 계산 (대각선으로부터의 거리)
    predicted = slope * theoretical_quantiles + intercept
    residuals = sample_quantiles_std - predicted
    rmse = np.sqrt(np.mean(residuals**2))
    
    # 패턴 분석
    # 치우침 감지: 양끝점들의 잔차 분석
    n_points = len(theoretical_quantiles)
    left_tail = residuals[:n_points//10]  # 왼쪽 10%
    right_tail = residuals[-n_points//10:]  # 오른쪽 10%
    
    skewness_pattern = "대칭적"
    if np.mean(left_tail) > 0.1 and np.mean(right_tail) < -0.1:
        skewness_pattern = "우측 치우침 (S자 곡선)"
    elif np.mean(left_tail) < -0.1 and np.mean(right_tail) > 0.1:
        skewness_pattern = "좌측 치우침 (역S자 곡선)"
    
    # 꼬리 두께 감지: 양끝에서 대각선으로부터의 거리
    tail_deviation = np.mean(np.abs(residuals[:n_points//20])) + np.mean(np.abs(residuals[-n_points//20:]))
    
    tail_pattern = "정상적"
    if tail_deviation > 0.2:
        if np.mean(np.abs(left_tail)) > np.mean(np.abs(residuals[n_points//3:2*n_points//3])):
            tail_pattern = "두꺼운 꼬리 (Heavy tail)"
        else:
            tail_pattern = "얇은 꼬리 (Light tail)"
    
    # 정규성 등급 결정
    if correlation > 0.99:
        normality_grade = "A+ (매우 우수)"
        normality_comment = "완벽에 가까운 정규분포입니다!"
    elif correlation > 0.98:
        normality_grade = "A (우수)"
        normality_comment = "정규분포에 매우 가깝습니다."
    elif correlation > 0.95:
        normality_grade = "B+ (양호)"
        normality_comment = "정규분포에 가까우나 약간의 편차가 있습니다."
    elif correlation > 0.90:
        normality_grade = "B (보통)"
        normality_comment = "정규분포와 어느 정도 차이가 있습니다."
    else:
        normality_grade = "C (개선 필요)"
        normality_comment = "정규분포에서 상당히 벗어납니다."
    
    return {
        'correlation': correlation,
        'slope': slope,
        'intercept': intercept,
        'rmse': rmse,
        'skewness_pattern': skewness_pattern,
        'tail_pattern': tail_pattern,
        'normality_grade': normality_grade,
        'normality_comment': normality_comment
    }

# 인터랙티브 중심극한정리 데모
@interact(
    distribution=widgets.Dropdown(
        options=[('균등분포', 'uniform'), ('지수분포', 'exponential'), 
                ('감마분포', 'skewed'), ('정규분포', 'normal'), ('이봉분포', 'bimodal')],
        value='uniform',
        description='원분포:'
    ),
    n_samples=IntSlider(min=5, max=200, step=5, value=30, description='표본크기 (n)'),
    n_simulations=widgets.fixed(1000)
)
def interactive_clt(distribution, n_samples, n_simulations):
    # 기존 출력 클리어
    from IPython.display import clear_output
    clear_output(wait=True)
    
    fig, sample_means, theo_mean, theo_std, qq_data = demonstrate_central_limit_theorem(
        distribution, n_samples, n_simulations
    )
    
    # Q-Q 플롯 분석
    qq_analysis = analyze_qq_plot(qq_data['theoretical_quantiles'], qq_data['sample_quantiles_std'])
    
    # 통계 요약
    actual_mean = np.mean(sample_means)
    actual_std = np.std(sample_means)
    
    print(f"""

🎯 중심극한정리 검증 결과:

📊 이론적 평균: {theo_mean:.3f}  |  실제 평균: {actual_mean:.3f}
📏 이론적 표준오차: {theo_std:.3f}  |  실제 표준편차: {actual_std:.3f}
📐 평균 오차: {abs(theo_mean - actual_mean):.3f}
📐 표준편차 오차: {abs(theo_std - actual_std):.3f}
✨ 정규성 검정 (Shapiro-Wilk): p-value = {stats.shapiro(sample_means)[1]:.3f}
{'🎉 정규분포에 가깝습니다!' if stats.shapiro(sample_means)[1] > 0.05 else '⚠️ 표본크기를 늘려보세요!'}

🔍 Q-Q 플롯 상세 분석:

📈 선형 상관도: {qq_analysis['correlation']:.4f} → {qq_analysis['normality_grade']}
💬 {qq_analysis['normality_comment']}

📊 패턴 분석:
   • 치우침: {qq_analysis['skewness_pattern']}
   • 꼬리 형태: {qq_analysis['tail_pattern']}
   • 대각선 적합도 (RMSE): {qq_analysis['rmse']:.3f}

🎓 Q-Q 플롯 해석 가이드:
   ✅ 상관도 > 0.99: 점들이 빨간 대각선에 거의 완벽하게 일치
   ✅ 상관도 > 0.95: 정규분포 가정이 합리적
   ⚠️ 상관도 < 0.95: 정규분포에서 벗어남, 표본크기 증가 고려
   
   📖 패턴별 의미:
   • S자 곡선: 원분포의 치우침이 아직 남아있음
   • 두꺼운 꼬리: 극값이 정규분포보다 자주 발생
   • 얇은 꼬리: 극값이 정규분포보다 드물게 발생

💡 중심극한정리 효과: {'강함 💪' if qq_analysis['correlation'] > 0.98 else '보통 ⚖️' if qq_analysis['correlation'] > 0.95 else '약함 ⚠️'}

""")
    
    fig.show()
    
    return None

interactive(children=(Dropdown(description='원분포:', options=(('균등분포', 'uniform'), ('지수분포', 'exponential'), ('감마…

## ⚡ 3. Z-분포와 표준화: 모든 것을 하나로

Z-분포(표준정규분포)는 **평균이 0, 표준편차가 1**인 특별한 정규분포입니다.

### 🔄 표준화 공식

어떤 정규분포 X ~ N(μ, σ²)든 다음 공식으로 표준화할 수 있습니다:

$$Z = \frac{X - \mu}{\sigma} \sim N(0, 1)$$

표본평균의 경우:

$$Z = \frac{\bar{X} - \mu}{\frac{\sigma}{\sqrt{n}}} \sim N(0, 1)$$

### 🎯 Z-분포의 특징
- **평균**: 0
- **표준편차**: 1
- **68-95-99.7 규칙**이 Z값으로 간단해짐:
  - 68%: -1 ≤ Z ≤ 1
  - 95%: -1.96 ≤ Z ≤ 1.96
  - 99%: -2.58 ≤ Z ≤ 2.58

In [4]:
# 🎨 Z-분포와 임계값 시각화
def plot_z_distribution_with_critical_values(alpha=0.05, test_type='two-tailed'):
    """Z-분포와 임계값을 시각화하는 함수"""
    
    # 고정된 범위 설정
    x_min, x_max = -4, 4
    y_min, y_max = 0, 0.45
    
    # Z값 범위
    z = np.linspace(x_min, x_max, 1000)
    pdf = stats.norm.pdf(z, 0, 1)
    
    # 임계값 계산
    if test_type == 'two-tailed':
        z_critical = stats.norm.ppf(1 - alpha/2)
        z_crit_neg = -z_critical
    elif test_type == 'right-tailed':
        z_critical = stats.norm.ppf(1 - alpha)
        z_crit_neg = None
    else:  # left-tailed
        z_critical = stats.norm.ppf(alpha)
        z_crit_neg = None
    
    # 플롯 생성
    fig = go.Figure()
    
    # 기본 Z-분포 곡선
    fig.add_trace(go.Scatter(
        x=z, y=pdf,
        mode='lines',
        name='표준정규분포 N(0,1)',
        line=dict(width=3, color='blue')
    ))
    
    # 기각역 표시
    if test_type == 'two-tailed':
        # 왼쪽 기각역
        z_left = z[z <= z_crit_neg]
        pdf_left = stats.norm.pdf(z_left, 0, 1)
        fig.add_trace(go.Scatter(
            x=np.concatenate([z_left, z_left[::-1]]),
            y=np.concatenate([pdf_left, np.zeros(len(pdf_left))]),
            fill='toself',
            fillcolor='rgba(255, 0, 0, 0.3)',
            line=dict(width=0),
            name=f'기각역 (α/2 = {alpha/2:.3f})',
            showlegend=True
        ))
        
        # 오른쪽 기각역
        z_right = z[z >= z_critical]
        pdf_right = stats.norm.pdf(z_right, 0, 1)
        fig.add_trace(go.Scatter(
            x=np.concatenate([z_right, z_right[::-1]]),
            y=np.concatenate([pdf_right, np.zeros(len(pdf_right))]),
            fill='toself',
            fillcolor='rgba(255, 0, 0, 0.3)',
            line=dict(width=0),
            showlegend=False
        ))
        
        # 임계값 선들
        for z_val, label in [(z_crit_neg, f'Z = {z_crit_neg:.3f}'), 
                            (z_critical, f'Z = {z_critical:.3f}')]:
            fig.add_vline(x=z_val, line_dash="dash", line_color="red",
                         annotation_text=label)
    
    elif test_type == 'right-tailed':
        z_tail = z[z >= z_critical]
        pdf_tail = stats.norm.pdf(z_tail, 0, 1)
        fig.add_trace(go.Scatter(
            x=np.concatenate([z_tail, z_tail[::-1]]),
            y=np.concatenate([pdf_tail, np.zeros(len(pdf_tail))]),
            fill='toself',
            fillcolor='rgba(255, 0, 0, 0.3)',
            line=dict(width=0),
            name=f'기각역 (α = {alpha:.3f})',
            showlegend=True
        ))
        
        fig.add_vline(x=z_critical, line_dash="dash", line_color="red",
                     annotation_text=f'Z = {z_critical:.3f}')
    
    else:  # left-tailed
        z_tail = z[z <= z_critical]
        pdf_tail = stats.norm.pdf(z_tail, 0, 1)
        fig.add_trace(go.Scatter(
            x=np.concatenate([z_tail, z_tail[::-1]]),
            y=np.concatenate([pdf_tail, np.zeros(len(pdf_tail))]),
            fill='toself',
            fillcolor='rgba(255, 0, 0, 0.3)',
            line=dict(width=0),
            name=f'기각역 (α = {alpha:.3f})',
            showlegend=True
        ))
        
        fig.add_vline(x=z_critical, line_dash="dash", line_color="red",
                     annotation_text=f'Z = {z_critical:.3f}')
    
    # 레이아웃 설정 - 축 범위 고정
    fig.update_layout(
        title=f'Z-분포와 임계값 ({test_type}, α = {alpha})',
        xaxis_title='Z 값',
        yaxis_title='확률밀도',
        template='plotly_white',
        height=500,
        xaxis=dict(range=[x_min, x_max]),  # X축 고정
        yaxis=dict(range=[y_min, y_max])   # Y축 고정
    )
    
    return fig

# 인터랙티브 Z-분포 시각화
@interact(
    alpha=FloatSlider(min=0.01, max=0.2, step=0.01, value=0.05, description='유의수준 (α)'),
    test_type=widgets.Dropdown(
        options=[('양측검정', 'two-tailed'), ('우측검정', 'right-tailed'), ('좌측검정', 'left-tailed')],
        value='two-tailed',
        description='검정 유형:'
    )
)
def interactive_z_distribution(alpha, test_type):
    # 기존 출력 클리어
    from IPython.display import clear_output
    clear_output(wait=True)
    
    fig = plot_z_distribution_with_critical_values(alpha, test_type)
    
    # 임계값 정보
    if test_type == 'two-tailed':
        z_crit = stats.norm.ppf(1 - alpha/2)
        print(f"""

🎯 임계값 정보 (양측검정):

📊 유의수준: α = {alpha}
📊 각 쪽 유의수준: α/2 = {alpha/2:.3f}
📏 임계값: ±{z_crit:.3f}
🔍 기각조건: |Z| > {z_crit:.3f}

""")
    elif test_type == 'right-tailed':
        z_crit = stats.norm.ppf(1 - alpha)
        print(f"""

🎯 임계값 정보 (우측검정):

📊 유의수준: α = {alpha}
📏 임계값: {z_crit:.3f}
🔍 기각조건: Z > {z_crit:.3f}

""")
    else:
        z_crit = stats.norm.ppf(alpha)
        print(f"""

🎯 임계값 정보 (좌측검정):

📊 유의수준: α = {alpha}
📏 임계값: {z_crit:.3f}
🔍 기각조건: Z < {z_crit:.3f}

""")
    
    fig.show()
    
    return None

interactive(children=(FloatSlider(value=0.05, description='유의수준 (α)', max=0.2, min=0.01, step=0.01), Dropdown(…

## 🧪 4. Z-검정: 완벽한 세상의 가설검정

Z-검정은 **모집단의 표준편차 σ를 알고 있을 때** 사용하는 가설검정 방법입니다.

### 🤔 먼저 이해하고 가자! - 자주 헷갈리는 개념들

#### ❓ **Q1: Z-분포는 모집단인가요?**
**A: 아닙니다!** Z-분포는 **검정통계량의 분포**입니다.

- **모집단**: 우리가 관심있는 실제 데이터들 (예: 모든 스마트폰 배터리 수명)
- **표본**: 모집단에서 뽑은 일부 데이터 (예: 36개 배터리)
- **Z-분포**: 표본에서 계산한 Z값들이 따르는 **이론적 분포** (평균 0, 표준편차 1)

```
🏭 실제 상황:
모집단 → 표본 추출 → Z값 계산 → Z-분포와 비교
```

#### ❓ **Q2: 왜 95%를 벗어난 게 문제인가요?**
**A: 이것이 바로 가설검정의 핵심입니다!**

**🎯 가설검정의 논리:**
1. **"배터리 수명이 24시간이다"**라고 가정 (귀무가설 H₀)
2. 만약 이게 참이라면, Z값은 95% 확률로 -1.96 ~ +1.96 사이에 있어야 함
3. 그런데 실제 계산한 Z값이 **-2.4**가 나옴 (95% 범위 밖!)
4. **"어? 이상하네? 정말로 24시간이 맞나?"** → 가설을 의심하게 됨

**🔍 기준이 95%인 이유:**
- **α = 0.05 (5%)**: 우리가 설정한 "오판할 확률"
- **95% = 1-α**: "올바른 판단을 할 확률"
- 만약 H₀가 참이라면, Z값이 95% 범위를 벗어날 확률은 단 5%
- 그런데 실제로 벗어났다? → H₀가 거짓일 가능성이 높다!

### 🌟 **왜 가설검정을 하는가? - 검정의 필요성**

#### **🤷‍♂️ 일상에서의 의사결정 문제**

**상황 1: 새로운 다이어트 방법** 🥗
- "이 방법으로 한 달에 평균 5kg 감량 가능!" (광고 문구)
- **문제**: 정말 효과가 있을까? 아니면 그냥 우연일까?
- **해결**: 실험해보고 통계적으로 검증해야 함

**상황 2: 새로운 온라인 광고** 📱
- A 광고: 클릭률 3.2%, B 광고: 클릭률 3.8%
- **문제**: B가 정말 더 좋을까? 아니면 그냥 운이 좋았을 뿐일까?
- **해결**: 충분한 데이터로 통계적 검정 필요

**상황 3: 신약 개발** 💊
- 기존 약 효과: 70%, 신약 효과: 75%
- **문제**: 5% 차이가 진짜 개선일까? 아니면 측정 오차일까?
- **해결**: 엄격한 통계적 검증 필수

#### **🎲 "우연"과 "진짜 효과"를 구분하는 방법**

**핵심 아이디어**: 
> **"만약 정말 차이가 없다면, 이런 극단적인 결과가 나올 확률은?"**

**예시**: 동전 던지기
- 동전을 10번 던져서 앞면이 8번 나옴
- **질문**: 이 동전이 불공정한가?
- **검정**: 공정한 동전이라면 8번 이상 앞면이 나올 확률 = 5.5%
- **결론**: 5.5% < 5% → 이 동전은 의심스럽다!

#### **📊 비즈니스에서의 실제 적용**

**A/B 테스트**: 웹사이트 버전 비교
- A버전: 구매율 2.1% (1000명 중 21명)
- B버전: 구매율 2.8% (1000명 중 28명)
- **단순 비교**: B가 더 좋아 보임
- **가설검정**: 이 차이가 **통계적으로 유의**한지 확인
- **결과**: p-값이 0.03 → B버전이 실제로 더 효과적!

**품질 관리**: 제품 불량률 검사
- 기준: 불량률 2% 이하
- 오늘 검사: 100개 중 4개 불량 (4%)
- **가설검정**: H₀: 불량률 = 2% vs H₁: 불량률 > 2%
- **결과**: 통계적으로 유의하다면 생산 라인 점검 필요

### 📋 Z-검정의 조건
1. **모집단이 정규분포**를 따르거나, **표본 크기가 충분히 큼** (n ≥ 30)
2. **모집단의 표준편차 σ가 알려져 있음** ⭐ (핵심 조건!)
3. 관찰값들이 **독립적**임

### 🎯 가설검정의 기본 개념

#### 🔄 **귀무가설 (H₀)과 대립가설 (H₁)**

**귀무가설 (Null Hypothesis, H₀)**:
- **"변화가 없다"**, **"차이가 없다"**, **"효과가 없다"**는 가설
- 우리가 **의심하고 있는** 주장
- 보통 **기존 상식**이나 **현재 상태**를 나타냄
- 예: H₀: μ = 24시간 (배터리 수명이 기준과 같다)

**대립가설 (Alternative Hypothesis, H₁)**:
- **"변화가 있다"**, **"차이가 있다"**, **"효과가 있다"**는 가설  
- 우리가 **증명하고 싶은** 주장
- 보통 **새로운 발견**이나 **변화**를 나타냄
- 예: H₁: μ ≠ 24시간 (배터리 수명이 기준과 다르다)

#### 🎭 **가설검정의 종류**

**1. 양측검정 (Two-tailed test)**: μ ≠ μ₀
- "기준과 **다르다**" (크거나 작거나)
- 기각역이 **양쪽**에 있음
- 예: 새 약이 기존 약과 **효과가 다른지** 알고 싶을 때

**2. 우측검정 (Right-tailed test)**: μ > μ₀  
- "기준보다 **크다**"
- 기각역이 **오른쪽**에만 있음
- 예: 새 방법이 기존 방법보다 **더 좋은지** 알고 싶을 때

**3. 좌측검정 (Left-tailed test)**: μ < μ₀
- "기준보다 **작다**"  
- 기각역이 **왼쪽**에만 있음
- 예: 새 처리법이 부작용을 **더 줄이는지** 알고 싶을 때

### 🎯 Z-검정 절차 (단계별 설명)

#### **1단계: 가설 설정**
```
H₀: μ = μ₀  (귀무가설 - 우리가 의심하는 주장)
H₁: μ ≠ μ₀  (대립가설 - 우리가 증명하고 싶은 주장)
```

#### **2단계: 유의수준 α 설정**
- **α = 0.05**: "5% 확률로 잘못 판단할 수 있다"
- **1-α = 0.95**: "95% 확률로 올바르게 판단한다"

#### **3단계: 검정통계량 계산**
$$Z = \frac{\bar{X} - \mu_0}{\frac{\sigma}{\sqrt{n}}}$$

**의미**: 
- 분자: 표본평균과 가설평균의 차이
- 분모: 표준오차 (불확실성의 크기)
- **Z값이 클수록**: 가설과 실제의 차이가 크다

#### **4단계: p-값 계산**
- **p-값**: H₀가 참일 때, 우리가 관찰한 것보다 극단적인 결과가 나올 확률
- **p < α**: "이런 극단적인 결과가 나올 확률이 5%보다 작다" → H₀ 기각
- **p ≥ α**: "이런 결과는 충분히 일어날 수 있다" → H₀ 채택

#### **5단계: 결론 도출**
- **H₀ 기각**: "통계적으로 유의한 차이가 있다"
- **H₀ 채택**: "통계적으로 유의한 차이가 없다"

### 🎪 **실제 예시로 이해하기**

**상황**: 스마트폰 배터리 수명 검사
- **기준**: 24시간 (μ₀ = 24)
- **표본**: 36개 배터리, 평균 23.2시간
- **모집단 표준편차**: σ = 2시간 (이미 알려진 값)

**1단계**: 가설 설정
- H₀: μ = 24 (배터리 수명이 기준과 같다)
- H₁: μ ≠ 24 (배터리 수명이 기준과 다르다)

**2단계**: α = 0.05 설정

**3단계**: Z값 계산
- Z = (23.2 - 24) / (2/√36) = -0.8 / 0.333 = **-2.4**

**4단계**: 판단
- |Z| = 2.4 > 1.96 → **기각역에 속함**
- p-값 ≈ 0.016 < 0.05 → **유의함**

**5단계**: 결론
- **H₀ 기각**: 배터리 수명이 24시간과 유의하게 다르다
- **실제 의미**: 이 배치는 품질 기준을 만족하지 않는다

### 💡 **핵심 통찰**

> **"95%를 벗어났다"**의 의미:
> 
> 만약 정말로 배터리 수명이 24시간이라면, 이렇게 극단적인 결과(Z = -2.4)가 나올 확률은 단 1.6%입니다. 
> 
> 이렇게 희귀한 일이 일어났다는 것은 **"24시간이라는 가설이 틀렸을 가능성이 높다"**는 의미입니다!

> **가설검정의 본질**:
> 
> 단순히 "다르다/같다"를 말하는 게 아니라, **"우연으로 설명하기에는 너무 극단적인 차이인가?"**를 과학적으로 판단하는 도구입니다!

Z-검정의 아름다움은 **완벽한 정보**(σ를 알고 있음)에 기반한다는 점입니다. 하지만 현실에서 σ를 정확히 아는 경우는 거의 없습니다! 😱

In [5]:
# 🧪 Z-검정 시뮬레이터
class ZTestSimulator:
    def __init__(self):
        self.results = {}
    
    def perform_z_test(self, sample_data, population_mean, population_std, 
                      alpha=0.05, test_type='two-tailed'):
        """Z-검정 수행"""
        
        n = len(sample_data)
        sample_mean = np.mean(sample_data)
        
        # 검정통계량 계산
        z_statistic = (sample_mean - population_mean) / (population_std / np.sqrt(n))
        
        # p-값 계산
        if test_type == 'two-tailed':
            p_value = 2 * (1 - stats.norm.cdf(abs(z_statistic)))
        elif test_type == 'right-tailed':
            p_value = 1 - stats.norm.cdf(z_statistic)
        else:  # left-tailed
            p_value = stats.norm.cdf(z_statistic)
        
        # 임계값 계산
        if test_type == 'two-tailed':
            z_critical = stats.norm.ppf(1 - alpha/2)
            rejection_condition = abs(z_statistic) > z_critical
        elif test_type == 'right-tailed':
            z_critical = stats.norm.ppf(1 - alpha)
            rejection_condition = z_statistic > z_critical
        else:
            z_critical = stats.norm.ppf(alpha)
            rejection_condition = z_statistic < z_critical
        
        # 신뢰구간 계산 (양측)
        margin_of_error = stats.norm.ppf(1 - alpha/2) * (population_std / np.sqrt(n))
        ci_lower = sample_mean - margin_of_error
        ci_upper = sample_mean + margin_of_error
        
        return {
            'sample_mean': sample_mean,
            'z_statistic': z_statistic,
            'p_value': p_value,
            'z_critical': z_critical,
            'reject_null': rejection_condition,
            'ci_lower': ci_lower,
            'ci_upper': ci_upper,
            'alpha': alpha,
            'test_type': test_type
        }
    
    def visualize_z_test(self, result, population_mean):
        """Z-검정 결과 시각화"""
        
        z = np.linspace(-4, 4, 1000)
        pdf = stats.norm.pdf(z, 0, 1)
        
        fig = go.Figure()
        
        # 기본 분포
        fig.add_trace(go.Scatter(
            x=z, y=pdf,
            mode='lines',
            name='표준정규분포',
            line=dict(width=3, color='blue')
        ))
        
        # 기각역 표시
        alpha = result['alpha']
        test_type = result['test_type']
        
        if test_type == 'two-tailed':
            z_crit = result['z_critical']
            
            # 좌측 기각역
            z_left = z[z <= -z_crit]
            pdf_left = stats.norm.pdf(z_left, 0, 1)
            fig.add_trace(go.Scatter(
                x=np.concatenate([z_left, z_left[::-1]]),
                y=np.concatenate([pdf_left, np.zeros(len(pdf_left))]),
                fill='toself',
                fillcolor='rgba(255, 0, 0, 0.3)',
                line=dict(width=0),
                name='기각역',
                showlegend=True
            ))
            
            # 우측 기각역
            z_right = z[z >= z_crit]
            pdf_right = stats.norm.pdf(z_right, 0, 1)
            fig.add_trace(go.Scatter(
                x=np.concatenate([z_right, z_right[::-1]]),
                y=np.concatenate([pdf_right, np.zeros(len(pdf_right))]),
                fill='toself',
                fillcolor='rgba(255, 0, 0, 0.3)',
                line=dict(width=0),
                showlegend=False
            ))
        
        # 검정통계량 표시
        z_stat = result['z_statistic']
        fig.add_vline(
            x=z_stat, 
            line_dash="solid", 
            line_color="green" if not result['reject_null'] else "red",
            line_width=3,
            annotation_text=f'Z = {z_stat:.3f}'
        )
        
        # 임계값 표시
        if test_type == 'two-tailed':
            for z_val in [-result['z_critical'], result['z_critical']]:
                fig.add_vline(x=z_val, line_dash="dash", line_color="orange")
        
        # 레이아웃
        decision = "귀무가설 기각" if result['reject_null'] else "귀무가설 채택"
        fig.update_layout(
            title=f'Z-검정 결과: {decision} (p = {result["p_value"]:.4f})',
            xaxis_title='Z 값',
            yaxis_title='확률밀도',
            template='plotly_white',
            height=500
        )
        
        return fig

# Z-검정 실습 예제
def z_test_example():
    """Z-검정 실습 예제: 공장 제품 품질 관리"""
    
    print("""

📦 사례연구: 스마트폰 배터리 수명 검사

🏭 상황:
- 스마트폰 배터리의 평균 수명은 24시간이어야 함 (품질 기준)
- 과거 데이터로부터 표준편차는 2시간으로 알려져 있음 (σ = 2)
- 새로운 생산 배치에서 36개 샘플을 추출하여 검사
- 검사 결과 평균 수명이 23.2시간으로 나타남
🤔 질문: 이 배치가 품질 기준을 만족한다고 볼 수 있을까?

""")
    
    # 데이터 설정
    np.random.seed(42)
    true_mean = 23.2  # 실제 새 배치의 평균
    population_std = 2
    sample_size = 36
    hypothesized_mean = 24  # 품질 기준
    
    # 샘플 데이터 생성 (실제 평균 23.2시간)
    sample_data = np.random.normal(true_mean, population_std, sample_size)
    
    # Z-검정 수행
    simulator = ZTestSimulator()
    result = simulator.perform_z_test(
        sample_data, hypothesized_mean, population_std, 
        alpha=0.05, test_type='two-tailed'
    )
    
    # 결과 시각화
    fig = simulator.visualize_z_test(result, hypothesized_mean)
    fig.show()
    
    # 결과 출력
    print(f"""

📊 Z-검정 결과 분석:

🎯 가설 설정:
    H₀: μ = 24시간 (품질 기준 만족)
    H₁: μ ≠ 24시간 (품질 기준 불만족)

📈 검정 통계량:
    표본 평균: {result['sample_mean']:.3f}시간
    표준오차: {population_std/np.sqrt(sample_size):.3f}시간
    Z 통계량: {result['z_statistic']:.3f}
    p-값: {result['p_value']:.4f}

🎯 임계값: ±{result['z_critical']:.3f}

📋 95% 신뢰구간: [{result['ci_lower']:.3f}, {result['ci_upper']:.3f}]


{'🚨 결론: 귀무가설을 기각합니다!' if result['reject_null'] else '✅ 결론: 귀무가설을 채택합니다!'}
{'   → 이 배치는 품질 기준을 만족하지 않습니다.' if result['reject_null'] else '   → 이 배치는 품질 기준을 만족합니다.'}

💡 해석:
{'   - p-값이 0.05보다 작으므로 통계적으로 유의한 차이가 있습니다.' if result['reject_null'] else '   - p-값이 0.05보다 크므로 통계적으로 유의한 차이가 없습니다.'}
{'   - 배터리 수명이 기준보다 유의하게 짧습니다.' if result['reject_null'] else '   - 배터리 수명이 기준과 유의한 차이가 없습니다.'}

""")

# 실습 실행
z_test_example()



📦 사례연구: 스마트폰 배터리 수명 검사

🏭 상황:
- 스마트폰 배터리의 평균 수명은 24시간이어야 함 (품질 기준)
- 과거 데이터로부터 표준편차는 2시간으로 알려져 있음 (σ = 2)
- 새로운 생산 배치에서 36개 샘플을 추출하여 검사
- 검사 결과 평균 수명이 23.2시간으로 나타남
🤔 질문: 이 배치가 품질 기준을 만족한다고 볼 수 있을까?






📊 Z-검정 결과 분석:

🎯 가설 설정:
    H₀: μ = 24시간 (품질 기준 만족)
    H₁: μ ≠ 24시간 (품질 기준 불만족)

📈 검정 통계량:
    표본 평균: 22.874시간
    표준오차: 0.333시간
    Z 통계량: -3.377
    p-값: 0.0007

🎯 임계값: ±1.960

📋 95% 신뢰구간: [22.221, 23.528]


🚨 결론: 귀무가설을 기각합니다!
   → 이 배치는 품질 기준을 만족하지 않습니다.

💡 해석:
   - p-값이 0.05보다 작으므로 통계적으로 유의한 차이가 있습니다.
   - 배터리 수명이 기준보다 유의하게 짧습니다.




## 🎯 5. 신뢰구간: 불확실성의 정량화

신뢰구간은 **모집단 모수의 가능한 범위**를 나타내는 구간입니다.

### 📏 95% 신뢰구간의 의미

> 동일한 방법으로 100번의 표본을 뽑아 신뢰구간을 구한다면, 그 중 약 95개의 구간이 실제 모집단 평균 μ를 포함할 것이다.

### 🔢 신뢰구간 공식 (Z-분포)

$$\bar{X} \pm Z_{\alpha/2} \cdot \frac{\sigma}{\sqrt{n}}$$

여기서:
- $\bar{X}$: 표본평균
- $Z_{\alpha/2}$: 임계값 (예: 95% 신뢰구간이면 Z₀.₀₂₅ = 1.96)
- $\frac{\sigma}{\sqrt{n}}$: 표준오차

### ⚠️ 흔한 오해

❌ **잘못된 해석**: "모집단 평균이 이 구간에 있을 확률이 95%이다"

✅ **올바른 해석**: "이와 같은 방법으로 구한 구간들 중 95%가 모집단 평균을 포함한다"

In [ ]:
# 🎯 신뢰구간 시뮬레이션
def confidence_interval_simulation(true_mean=50, true_std=10, sample_size=30, 
                                 confidence_level=0.95, n_samples=100):
    """신뢰구간의 의미를 시뮬레이션으로 보여주는 함수"""
    
    np.random.seed(42)
    alpha = 1 - confidence_level
    z_critical = stats.norm.ppf(1 - alpha/2)
    
    # 여러 표본에서 신뢰구간들 계산
    sample_means = []
    ci_lowers = []
    ci_uppers = []
    contains_true_mean = []
    
    for i in range(n_samples):
        # 표본 추출
        sample = np.random.normal(true_mean, true_std, sample_size)
        sample_mean = np.mean(sample)
        
        # 신뢰구간 계산
        margin_of_error = z_critical * (true_std / np.sqrt(sample_size))
        ci_lower = sample_mean - margin_of_error
        ci_upper = sample_mean + margin_of_error
        
        sample_means.append(sample_mean)
        ci_lowers.append(ci_lower)
        ci_uppers.append(ci_upper)
        contains_true_mean.append(ci_lower <= true_mean <= ci_upper)
    
    # 결과 정리
    coverage_rate = np.mean(contains_true_mean)
    
    # 시각화 - 축 범위 고정
    fig = go.Figure()
    
    # 고정된 X축 범위 설정 (더 넓은 범위)
    x_min, x_max = 20, 80
    
    # 신뢰구간들 그리기 - 모든 n_samples 표시하되 너무 많으면 제한
    display_count = min(200, n_samples)  # 최대 100개까지 표시
    
    for i in range(display_count):
        color = 'green' if contains_true_mean[i] else 'red'
        
        # 신뢰구간 선
        fig.add_trace(go.Scatter(
            x=[ci_lowers[i], ci_uppers[i]],
            y=[i, i],
            mode='lines',
            line=dict(color=color, width=2),
            showlegend=False
        ))
        
        # 표본평균 점
        fig.add_trace(go.Scatter(
            x=[sample_means[i]],
            y=[i],
            mode='markers',
            marker=dict(color=color, size=6),
            showlegend=False
        ))
    
    # 실제 모집단 평균 선
    fig.add_vline(
        x=true_mean,
        line_dash="dash",
        line_color="blue",
        line_width=3,
        annotation_text=f'실제 평균 = {true_mean}'
    )
    
    # 범례용 더미 트레이스
    fig.add_trace(go.Scatter(
        x=[None], y=[None],
        mode='lines',
        line=dict(color='green', width=2),
        name=f'포함 구간 ({np.sum(contains_true_mean[:display_count])}/{display_count})'
    ))
    
    fig.add_trace(go.Scatter(
        x=[None], y=[None],
        mode='lines',
        line=dict(color='red', width=2),
        name=f'미포함 구간 ({display_count - np.sum(contains_true_mean[:display_count])}/{display_count})'
    ))
    
    # 레이아웃 - Y축 범위를 동적으로 조정
    fig.update_layout(
        title=f'{confidence_level*100:.0f}% 신뢰구간 시뮬레이션 (커버리지: {coverage_rate:.1%}, 표시: {display_count}/{n_samples})',
        xaxis_title='값',
        yaxis_title='표본 번호',
        template='plotly_white',
        height=max(400, min(800, display_count * 8)),  # 동적 높이 조정
        yaxis=dict(autorange="reversed", range=[-2, display_count+2]),  # Y축을 display_count에 맞춤
        xaxis=dict(range=[x_min, x_max])  # X축 고정
    )
    
    return fig, coverage_rate, sample_means, ci_lowers, ci_uppers

# 인터랙티브 신뢰구간 시뮬레이션
@interact(
    confidence_level=FloatSlider(min=0.8, max=0.99, step=0.01, value=0.95, 
                                description='신뢰수준'),
    sample_size=IntSlider(min=10, max=100, step=10, value=30, 
                         description='표본크기'),
    n_samples=IntSlider(min=50, max=200, step=50, value=100, 
                       description='시뮬레이션 횟수')
)
def interactive_ci_simulation(confidence_level, sample_size, n_samples):
    # 기존 출력 클리어
    from IPython.display import clear_output
    clear_output(wait=True)
    
    fig, coverage_rate, _, _, _ = confidence_interval_simulation(
        true_mean=50, true_std=10, sample_size=sample_size, 
        confidence_level=confidence_level, n_samples=n_samples
    )
    
    expected_coverage = confidence_level
    difference = abs(coverage_rate - expected_coverage)
    
    print(f"""

📊 신뢰구간 분석 결과:

🎯 이론적 커버리지: {expected_coverage:.1%}
📈 실제 커버리지: {coverage_rate:.1%}
📐 차이: {difference:.1%}

💡 해석:
{f'✅ 이론과 잘 일치합니다!' if difference < 0.05 else '⚠️ 시뮬레이션 횟수를 늘려보세요.'}

🔍 신뢰구간의 의미:
- 초록선: 실제 평균을 포함하는 신뢰구간
- 빨간선: 실제 평균을 포함하지 않는 신뢰구간
- 파란 점선: 실제 모집단 평균 (μ = 50)

ℹ️ 참고: 성능상 최대 100개의 신뢰구간만 시각화됩니다.

""")
    
    fig.show()
    
    return None

interactive(children=(FloatSlider(value=0.95, description='신뢰수준', max=0.99, min=0.8, step=0.01), IntSlider(val…

## 🤔 6. 완벽한 세상의 한계

지금까지 우리는 **완벽한 세상**에서의 통계 검정을 살펴봤습니다. 하지만 현실은 어떨까요?

### ❓ 생각해보기

다음 상황들을 생각해보세요:

1. **신제품 개발**: 새로운 약물의 효과를 테스트할 때, 과거 데이터가 없어서 σ를 모른다면?

2. **품질 관리**: 소규모 공장에서 매일 3-4개의 제품만 검사할 수 있다면?

3. **A/B 테스트**: 웹사이트 방문자 중 일부만 새 버전을 테스트할 때, 표본이 작다면?

### 🚨 Z-검정의 현실적 한계

1. **σ를 아는 경우가 드물다** 😱
   - 실제로는 표본표준편차 s를 사용해야 함
   - s는 σ의 추정값이므로 불확실성이 추가됨

2. **작은 표본 크기** 📏
   - 중심극한정리가 잘 작동하지 않음 (n < 30)
   - 표본분포가 정규분포에서 벗어남

3. **추정의 불확실성** 🎲
   - σ를 s로 대체하면 추가적인 변동성 발생
   - Z-분포보다 "더 넓은" 분포가 필요함

### 💡 다음 이야기 예고

> **1908년 더블린**, 기네스 맥주 공장의 한 젊은 통계학자가 이 문제로 고민하고 있었습니다. 그의 이름은 **William Gosset**... 그리고 그는 곧 통계학 역사를 바꿀 발견을 하게 됩니다! 🍺✨

다음 노트북에서는 이 흥미진진한 이야기와 함께 **t-분포의 탄생**을 다뤄보겠습니다!

In [ ]:
# 🎭 Z-검정 vs 실제 상황 비교 시뮬레이션
def compare_z_vs_reality(true_mean=100, true_std=15, sample_sizes=[5, 10, 30, 100]):
    """Z-검정(σ 알려짐)과 실제 상황(σ 모름) 비교"""
    
    np.random.seed(42)
    
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[f'n = {n}' for n in sample_sizes],
        specs=[[{'type': 'histogram'}, {'type': 'histogram'}],
               [{'type': 'histogram'}, {'type': 'histogram'}]]
    )
    
    results = {}
    
    # 고정된 축 범위 설정
    x_min, x_max = -4, 4
    y_min, y_max = 0, 250  # 히스토그램 최대 높이
    
    for idx, n in enumerate(sample_sizes):
        row = idx // 2 + 1
        col = idx % 2 + 1
        
        # 1000번의 표본 추출
        z_statistics = []
        t_statistics = []
        
        for _ in range(1000):
            sample = np.random.normal(true_mean, true_std, n)
            sample_mean = np.mean(sample)
            sample_std = np.std(sample, ddof=1)  # 표본표준편차
            
            # Z-통계량 (σ 알려진 경우 - 완벽한 세상)
            z_stat = (sample_mean - true_mean) / (true_std / np.sqrt(n))
            z_statistics.append(z_stat)
            
            # t-통계량 (σ 모르는 경우 - 현실 세상)
            t_stat = (sample_mean - true_mean) / (sample_std / np.sqrt(n))
            t_statistics.append(t_stat)
        
        # Z-통계량 히스토그램 (고정된 범위로)
        fig.add_trace(
            go.Histogram(
                x=z_statistics,
                nbinsx=50,
                name=f'Z-통계량 (n={n})',
                opacity=0.7,
                marker_color='blue',
                legendgroup=f'group{n}',
                xbins=dict(start=x_min, end=x_max, size=0.16)  # 고정된 bin 범위
            ),
            row=row, col=col
        )
        
        # t-통계량 히스토그램 (고정된 범위로)
        fig.add_trace(
            go.Histogram(
                x=t_statistics,
                nbinsx=50,
                name=f't-통계량 (n={n})',
                opacity=0.5,
                marker_color='red',
                legendgroup=f'group{n}',
                xbins=dict(start=x_min, end=x_max, size=0.16)  # 고정된 bin 범위
            ),
            row=row, col=col
        )
        
        # 이론적 분포 곡선 추가
        x_range = np.linspace(x_min, x_max, 100)
        
        # 표준정규분포 (Z) - 스케일링 조정
        y_norm = stats.norm.pdf(x_range, 0, 1) * 40  # 고정된 스케일링
        fig.add_trace(
            go.Scatter(
                x=x_range,
                y=y_norm,
                mode='lines',
                name=f'표준정규분포 (n={n})',
                line=dict(color='blue', width=3),
                legendgroup=f'group{n}'
            ),
            row=row, col=col
        )
        
        # t-분포 - 스케일링 조정
        y_t = stats.t.pdf(x_range, n-1) * 40  # 고정된 스케일링, 자유도 n-1
        fig.add_trace(
            go.Scatter(
                x=x_range,
                y=y_t,
                mode='lines',
                name=f't-분포 (df={n-1})',
                line=dict(color='red', width=3, dash='dash'),
                legendgroup=f'group{n}'
            ),
            row=row, col=col
        )
        
        # 통계 저장
        results[n] = {
            'z_mean': np.mean(z_statistics),
            'z_std': np.std(z_statistics),
            't_mean': np.mean(t_statistics),
            't_std': np.std(t_statistics)
        }
    
    # 레이아웃 업데이트 - 모든 서브플롯의 축 범위 고정
    fig.update_layout(
        title='완벽한 세상 vs 현실: σ를 알 때와 모를 때의 차이',
        height=800,
        showlegend=True
    )
    
    # 모든 서브플롯의 축 범위 고정
    for i in range(1, 3):  # rows
        for j in range(1, 3):  # cols
            fig.update_xaxes(range=[x_min, x_max], row=i, col=j)
            fig.update_yaxes(range=[y_min, y_max], row=i, col=j)
    
    return fig, results

# 비교 시뮬레이션 실행
fig, results = compare_z_vs_reality()
fig.show()

print("이 시뮬레이션이 보여주는 것:")
print()
print("두 가지 상황 비교:")
print("완벽한 세상 (Z-통계량):")
print("- 모집단 표준편차 σ = 15를 정확히 알고 있음")
print("- Z = (표본평균 - 모집단평균) / (σ / √n)")
print("- 파란색 히스토그램과 파란색 실선 (표준정규분포)")
print()
print("현실 세상 (t-통계량):")
print("- 모집단 표준편차 σ를 모르기 때문에 표본표준편차 s로 대체")
print("- t = (표본평균 - 모집단평균) / (s / √n)  ← 여기서 s는 매번 다름!")
print("- 빨간색 히스토그램과 빨간색 점선 (t-분포)")
print()
print("왜 t-통계량이 더 퍼져있을까?")
print("핵심 아이디어: σ를 모르니까 s로 추정해야 하는데, s는 추정값이라 불확실함이 추가됨!")
print()
print("수치 분석 결과:")

for n in [5, 10, 30, 100]:
    r = results[n]
    print(f"표본크기 n = {n}:")
    print(f"   Z-통계량 (완벽한 세상): 평균 = {r['z_mean']:.3f}, 표준편차 = {r['z_std']:.3f}")
    print(f"   t-통계량 (현실 세상):   평균 = {r['t_mean']:.3f}, 표준편차 = {r['t_std']:.3f}")
    print(f"   차이: {abs(r['t_std'] - r['z_std']):.3f} {'← 거의 없음' if abs(r['t_std'] - r['z_std']) < 0.1 else '← 상당한 차이!'}")
    print()

print("핵심 통찰:")
print("표본크기가 작을 때 (n=5, 10):")
print("- t-통계량이 Z-통계량보다 훨씬 더 퍼져있음 (표준편차가 큼)")
print("- 왜? s가 σ의 부정확한 추정이기 때문")
print("- 적은 데이터로는 σ를 잘 추정할 수 없음")
print()
print("표본크기가 클 때 (n=100):")
print("- t-통계량과 Z-통계량이 거의 비슷해짐")
print("- 왜? 큰 표본에서는 s가 σ에 매우 가까워짐 (대수의 법칙)")
print("- s ≈ σ이므로 t ≈ Z")
print()
print("William Gosset의 발견 (1908년):")
print("σ를 모를 때는 Z-분포 대신 더 넓은 분포(t-분포)를 써야 한다!")
print()
print("실제 문제:")
print("- 기네스 맥주 공장에서 품질 검사를 할 때")
print("- 매일 4-5개 맥주만 검사 가능 (작은 표본!)")
print("- σ를 정확히 알 수 없음")
print("- Z-검정을 쓰면 잘못된 결론을 내릴 위험")
print()
print("그래프에서 관찰:")
print("- n=5일 때: 빨간색(t-분포)이 파란색(Z-분포)보다 훨씬 넓음")
print("- n=100일 때: 빨간색과 파란색이 거의 겹침")
print()
print("결론:")
print("표본크기가 작고 σ를 모르는 현실적 상황에서는 t-분포를 사용해야 올바른 통계적 추론이 가능합니다!")
print()
print("다음 노트북에서 계속...")
print("→ Gosset이 어떻게 t-분포를 유도했는지 직접 경험해보기!")

## 📚 핵심 개념 요약

### ✨ 오늘 배운 것들

1. **정규분포**: 자연의 가장 아름다운 분포, 68-95-99.7 규칙

2. **중심극한정리**: 어떤 분포든 표본평균들은 정규분포에 수렴

3. **Z-분포**: 평균 0, 표준편차 1인 표준정규분포

4. **Z-검정**: σ를 알 때 사용하는 완벽한 가설검정

5. **신뢰구간**: 모수의 가능한 범위를 나타내는 구간

6. **현실의 한계**: σ를 모르는 상황의 문제점

### 🔑 핵심 공식들

- **표준화**: $Z = \frac{X - \mu}{\sigma}$
- **표본평균 분포**: $\bar{X} \sim N(\mu, \frac{\sigma^2}{n})$
- **Z-검정 통계량**: $Z = \frac{\bar{X} - \mu_0}{\frac{\sigma}{\sqrt{n}}}$
- **신뢰구간**: $\bar{X} \pm Z_{\alpha/2} \cdot \frac{\sigma}{\sqrt{n}}$

---

## 🧩 연습 문제

### 문제 1: 기본 개념
정규분포 N(100, 15²)에서 표본 25개를 뽑았을 때, 표본평균의 분포는?

**필요한 공식**: 표본평균의 분포는 $\bar{X} \sim N(\mu, \frac{\sigma^2}{n})$

### 문제 2: Z-검정 
어떤 공장의 제품 무게는 평균 500g, 표준편차 20g인 정규분포를 따른다고 알려져 있습니다. 새로운 생산 방식으로 만든 제품 36개의 평균 무게가 495g이었습니다. 이 차이가 통계적으로 유의한가요? (α = 0.05)

**필요한 정보**:
- Z-검정 통계량: $Z = \frac{\bar{X} - \mu_0}{\frac{\sigma}{\sqrt{n}}}$
- α = 0.05 양측검정 임계값: ±1.96
- 판정기준: |Z| > 1.96이면 귀무가설 기각

### 문제 3: 신뢰구간
위 문제에서 새로운 생산 방식의 평균 무게에 대한 95% 신뢰구간을 구하세요.

**필요한 정보**:
- 95% 신뢰구간 공식: $\bar{X} \pm Z_{0.025} \cdot \frac{\sigma}{\sqrt{n}}$
- Z₀.₀₂₅ = 1.96 (95% 신뢰구간 임계값)
- 표본평균: x̄ = 495g, σ = 20g, n = 36
- 해석: 신뢰구간에 가설값(500)이 포함되면 귀무가설과 일치

---

## 🚀 다음 여행지: "기네스의 비밀"

다음 노트북에서는 1908년 더블린로 시간여행을 떠납니다! 🍺

- William Gosset의 고민
- 작은 표본의 비밀
- Student's t-분포의 탄생
- 몬테카를로로 직접 유도해보기

**다음 노트북**: `02_birth_of_t_distribution.ipynb`